In [1]:
# Modified from: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb#scrollTo=XsysvFz1KqN0

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [3]:
%env UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT

env: UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT


In [4]:
from unsloth import FastLanguageModel, FastModel, UnslothTrainer, UnslothTrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "allenai/OLMo-2-1124-7B",
    max_seq_length = 2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.12.1: Fast Olmo2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha = 32,
    qat_scheme = "int4",
)

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
dataset = load_dataset("roneneldan/TinyStories", split = "train[:2500]")
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [8]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        logging_steps = 1,
        optim = "adamw_torch_4bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2500 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,500 | Num Epochs = 1 | Total steps = 157
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 39,976,960 of 7,338,594,304 (0.54% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.466100
2,1.588500
3,1.514400
4,1.439500
5,1.540100
6,1.525700
7,1.641900
8,1.364000
9,1.423500
10,1.451900


In [10]:
model.save_pretrained("qlora_olmo2")

In [11]:
model.save_pretrained_merged("unsloth_olmo2", tokenizer,)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 6 files from cache to `unsloth_olmo2`: 100%|██████████| 6/6 [01:12<00:00, 12.17s/it]


Successfully copied all 6 files from cache to `unsloth_olmo2`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 6/6 [01:51<00:00, 18.61s/it]


Unsloth: Merge process complete. Saved to `/content/unsloth_olmo2`


In [12]:
!git clone https://github.com/ggml-org/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 71321, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 71321 (delta 179), reused 95 (delta 95), pack-reused 71029 (from 2)
Receiving objects: 100% (71321/71321), 230.48 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (51446/51446), done.


In [13]:
%cd llama.cpp

/content/llama.cpp


In [14]:
!cmake -B build
!cmake --build build --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [15]:
!python3 convert_hf_to_gguf.py ../unsloth_olmo2 --outfile olmo2_quantized.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: unsloth_olmo2
INFO:hf-to-gguf:Model architecture: Olmo2ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00006.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00004-of-00006.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00005-of-00006.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00006-of-00006.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00006.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00006.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.10.ffn_down.weight,            torch.float32 --> Q8_0, shape = {11008, 4096}
INFO:hf-to-gguf:blk.10.ffn_gate.weight,            torch.float32 --> Q8_0, shape = {4096, 11008}
INFO:hf-to-gguf:blk.10.ffn_up.weight,              torch.fl

In [16]:
from google.colab import files
files.download('olmo2_quantized.gguf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>